In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
ls

artifacts/        dvc.yaml  params.yaml       setup.py     venv/
Chicken_project/  LICENSE   README.md         src/
config/           logs/     requirements.txt  template.py
data.zip          main.py   research/         templates/


# Entity

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    trained_model_path: Path
    dataframe_path: Path
    test_batch_size: int
    batch_size: int
    img_size: list
    channels: int
    color: str
    

# Config

In [14]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        config = self.config.evaluation
        eval_config = EvaluationConfig(
            trained_model_path=config.trained_model_path,
            dataframe_path=config.dataframe_path,
            test_batch_size=self.params.test_batch_size,
            batch_size=self.params.batch_size,
            img_size=self.params.img_size,
            channels=self.params.channels,
            color=self.params.color
        )
        return eval_config

In [33]:
ConfigurationManager().get_evaluation_config()

[2023-12-10 14:30:38,383: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-10 14:30:38,386: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 14:30:38,388: INFO: common: created directory at: artifacts]


EvaluationConfig(trained_model_path='artifacts/training/trained_model.keras', dataframe_path='artifacts/training', test_batch_size=40, batch_size=40, img_size=BoxList([224, 224]), channels=3, color='rgb')

# Component

In [34]:
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

In [43]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.ts_length = None
        
    def load_train_valid_test_df(self):
          # load generators
        path = os.path.join(self.config.dataframe_path,'train_df.pkl')
        with open(path, 'rb') as f:
            train_df = pickle.load(f)

        path = os.path.join(self.config.dataframe_path,'valid_df.pkl')
        with open(path, 'rb') as f:
            valid_df = pickle.load(f)

        path = os.path.join(self.config.dataframe_path,'test_df.pkl')
        with open(path, 'rb') as f:
            test_df = pickle.load(f)

        self.ts_length = len(test_df)

        return train_df, valid_df, test_df

        
    def create_train_valid_test_generator(self, train_df, valid_df, test_df):
        '''
        This function takes train, validation, and test dataframe and fit them into image data generator, because model takes data from image data generator.
        Image data generator converts images into tensors. 
        '''

        # define model parameters
        batch_size = self.config.batch_size
        img_size = self.config.img_size
        channels = self.config.channels
        color = self.config.color
        img_shape = (img_size[0], img_size[1], channels)
    
        ts_length = len(test_df)
        test_batch_size = self.config.batch_size
        test_steps = ts_length // test_batch_size
        # This function which will be used in image data generator for data augmentation, it just take the image and return it again.
        def scalar(img):
            return img
    
        tr_gen = ImageDataGenerator(preprocessing_function= scalar, horizontal_flip= True)
        ts_gen = ImageDataGenerator(preprocessing_function= scalar)
    
        train_gen = tr_gen.flow_from_dataframe(train_df, x_col = 'filepaths', y_col = 'labels', target_size= img_size, class_mode = "binary",
                                              color_mode= color, shuffle= False, batch_size= batch_size)
        valid_gen = ts_gen.flow_from_dataframe(valid_df, x_col = 'filepaths', y_col = 'labels', target_size= img_size, class_mode = "binary",
                                              color_mode= color, shuffle= False, batch_size= batch_size)
         # Note: we will use custom test_batch_size, and make shuffle= false
        test_gen = ts_gen.flow_from_dataframe(test_df, x_col = 'filepaths', y_col = 'labels', target_size= img_size, class_mode = "binary",
                                             color_mode= color, shuffle= False, batch_size= test_batch_size)
        self.train_gen = train_gen
        self.valid_gen = valid_gen
        self.test_gen = test_gen
        
        return train_gen, valid_gen, test_gen

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        model = self.load_model(self.config.trained_model_path)
        
        ts_length = self.ts_length
        test_batch_size = self.config.batch_size
        test_steps = ts_length // test_batch_size
        
        train_score = model.evaluate(self.train_gen, steps= test_steps, verbose= 1)
        valid_score = model.evaluate(self.valid_gen, steps= test_steps, verbose= 1)
        test_score = model.evaluate(self.test_gen, steps= test_steps, verbose= 1)
        
        print("Train Loss: ", train_score[0])
        print("Train Accuracy: ", train_score[1])
        print('-' * 20)
        print("Validation Loss: ", valid_score[0])
        print("Validation Accuracy: ", valid_score[1])
        print('-' * 20)
        print("Test Loss: ", test_score[0])
        print("Test Accuracy: ", test_score[1])

    
        scores = {"Train Loss": train_score[0], "Train Accuracy:": train_score[1],
                 "Validation Loss": valid_score[0], "Validation Accuracy:": valid_score[1],
                 "Test Loss": test_score[0], "Test Accuracy:": test_score[1]}
        
        save_json(path=Path("scores.json"), data=scores)

In [46]:
config = ConfigurationManager()
val_config = config.get_evaluation_config()
evaluation = Evaluation(val_config)
train_df, valid_df, test_df = evaluation.load_train_valid_test_df()
evaluation.create_train_valid_test_generator(train_df, valid_df, test_df)
evaluation.evaluation()

[2023-12-10 14:37:00,463: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-10 14:37:00,467: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 14:37:00,468: INFO: common: created directory at: artifacts]
Found 3904 validated image filenames belonging to 2 classes.
Found 488 validated image filenames belonging to 2 classes.
Found 488 validated image filenames belonging to 2 classes.
12/12 [==============================] - 3s 259ms/step - loss: 0.1858 - accuracy: 0.9438
Train Loss:  0.0483812615275383
Train Accuracy:  0.9895833134651184
--------------------
Validation Loss:  0.07457593083381653
Validation Accuracy:  0.9708333611488342
--------------------
Test Loss:  0.18583659827709198
Test Accuracy:  0.9437500238418579
[2023-12-10 14:37:12,054: INFO: common: json file saved at: scores.json]


In [ ]:
# Train Loss:  0.03094664216041565
# Train Accuracy:  0.9937499761581421
# --------------------
# Validation Loss:  0.07457593083381653
# Validation Accuracy:  0.9708333611488342
# --------------------
# Test Loss:  0.18583659827709198
# Test Accuracy:  0.9437500238418579